### Welcome to the first class on E-ID in this 3-class series
Before we start, make sure you have everything installed by following one of the outlined methods in the ReadMe file. 

### Purpose of this exercise
This jupyter notebook explains the following:
1. Basic E-ID example using RSA cryptographic scheme
2. Selective Disclosure using RSA and its downsides
3. What is unlinkability in the E-ID world
4. The previous points should explain to a degree the motivation behind the need for a more specialized cryptographic scheme

In [1]:
// We start by creating a typical E-ID object that we will use through out this exercise
const ID_DATA = {
    name: "Jack Sparrow",
    age: 61,
    profession: "IT Manager"
}

## Definition: Verifiable Credentials
A verifiable credential, in its simplest form, is a signed string of data. An issuer will issue a credential by signing a specific string of data then sharing that string of data along with a cryptographic signature that can prove that this string was authorized/ issued by this specific issuer.

In [2]:
import * as crypto from 'crypto';

### Issuer

In [3]:
/* 
 For an issuer to be able to start issuing Verifiable Credentials, it first needs
 to have its own cryptographic key pair. 
 Issuers will sign the data using their private key. 
*/
const { publicKey, privateKey } = crypto.generateKeyPairSync('rsa', {
    modulusLength: 4096,
    publicKeyEncoding: {
      type: 'spki',   
    
      format: 'pem'
    },
    privateKeyEncoding: {
      type: 'pkcs8',
      format: 'pem',
    }
});

In [18]:
// An issuer will convert the data object into a string
const message = JSON.stringify(ID_DATA)

// then create a signature for that data
const signer = crypto.createSign('SHA256');
signer.update(message);
const signature = signer.sign(privateKey, 'base64')

console.log(message);
console.log("----------------------------")
console.log(signature);

{"name":"Jack Sparrow","age":61,"profession":"IT Manager"}
----------------------------
GSQWERQsSGhciaKRqmuHY/GGUUIoqzVxomw3Otq/d1GHPEu1Y9KRaaL7ipS8tnqRLFv+MVe5o2oUh5hLqFr/gEwKLur/2qB9IKyiakSUl+XxlDOT+gYxQJduIy+yhzqJhHcplvA5obN+7JrajV8HzCCGQeWYRNJzaSFwuESFe9MGCqBbHd9d6NbbG3MZXFBQ6FyTCUgZxWjRwSidPE1Vzehywx/C+NChIm8PrpEPOtCLYMNqE4SKA5/lWyCV27gIT9QLZ+Hb8WQfzISZNGr2dWYpdZSDTZy6qjz9Gf9HPdys1MKJKSzNk5F4fAP9bXW3Ii4prEuvd2uWiRsd5fHXWPmLuX0qC/3GrawgJc3ioNW3o+UeiGc0qhbGP177liGVVuT1Ldkb8492PrpDzEzDMrSlwYfE6utKwm2LhpnaFR1a0QRz9BTzv4R+D5kCokxOMiChuso+7609INPublHpRSzJWnUkW1+gbwXHbLKfZdkZ5k9XThT0Q44h1lc3QpiNSuV+OzYS1vZUOpIhAAWhPMloxv938x+eA/GSwk0Cb58dpa4Ll+XA145e5P2Sy3IX3MZCLaZSsy+gNeYfDUXN7yk6l+NWgdE42mHdkul/rDsbEMad09TShweBVyRGKm937ZvnRVrsa4POrrXZo2/CKjTWG/HzFsAg0DkZmfoCPNo=


### Issuer →  Holder

After the signature is created, the data along with the signature is transfered to the holder

In this case, the holder can only do one thing with this data which is to share the whole data string along with the signature

### Holder →  Verifier

Once the data is sent from holder to verifier, the verifier can verify that information as follows:

In [10]:
// verifier now recieves "message" and "signature" data
// The verifier should also have access to the publicKey of the issuer

let verifier = crypto.createVerify('SHA256');
verifier.update(message)
console.log(verifier.verify(publicKey, signature, 'base64'))

true


In [11]:
// What happens if the data is not accurate or true?

verifier = crypto.createVerify('SHA256');
verifier.update("Another random message!")
console.log(verifier.verify(publicKey, signature, 'base64'))

false


---

## Selective disclosure

What if the holder of the credential wants to only share his name and profession but not his age?

The current implementation wouldn't allow for that, so we will need to change it to some extent.

In [42]:
// One way to implement selective disclosure is to hash every value, and the 
// holder of the credential would share only the values he intends to. 

// issuer will hash the values before signing the data
function hashValue(value: string, algorithm: string): string {
  const hash = crypto.createHash(algorithm);
  hash.update(value);
  return hash.digest('hex');
}
CHOSEN_HASH_ALGORITHM = 'sha256'
const ID_DATA_HASHED_VERSION = {
    name: hashValue("Jack Sparrow", CHOSEN_HASH_ALGORITHM),
    age: hashValue("61", CHOSEN_HASH_ALGORITHM),
    profession: hashValue("IT Manager", CHOSEN_HASH_ALGORITHM)
}

// An issuer will convert the data object into a string
const message = JSON.stringify(ID_DATA_HASHED_VERSION);

// then create a signature for that data
const signer = crypto.createSign('SHA256');
signer.update(message);

const signature = signer.sign(privateKey, 'base64');
console.log(message);
console.log("----------------------------");
console.log(signature);

{"name":"a5d0dd92108bc55cadfaf56e36a60a7121b059d9e0d09c2d24d853e9d1d14ea7","age":"d029fa3a95e174a19934857f535eb9427d967218a36ea014b70ad704bc6c8d1c","profession":"c249aac9d89f89626fbcaa519d53e386478a22b775414c324e61289e033f1652"}
----------------------------
mEZc+T3qjqlUlXm5d+srtgOYegk+WOHM56NYDlw9KncnuWb62cqfkALk9YX+IpDEFxWkJHavbE2++T7pyiEfXcG8HBTanC3bCHW8UtyNXhQupgkQw24/CplK2BN95CnmF33b7uU73Q5joyOcJAGMrIxtQN76+m5ky2RLs504IbH5OUqqNYn9P+UaTWY5jU0BNikWf8IqnsvNBsFy1YAaQoAfJU5xZGe4b8lhfWxKrObkrijCBKD+VDIjdXb9SoFH2JazQFeAZCtzVzq7yiefL8z4O+d7NLA0FpXFoK2nwHLlh4R2hNcLuBtT8nYZ0IZpA1BMgqC3laOrI86Jca8cqZW9uX5G5XilX6F7MyQjQ5NCc5NugkZcSb05cmgFKuCLF6ErnScaL4WGx4zowfPr37JH4GBlPq6+dhmx6ABJMzsYtHTpeUUz1nfzOjpDIxFd/GjHD9DkXzlXxOJuATn/auN29nQZGWKb13ZEWAnUzrE2BMFjNuxfNxzdBeLh6sL24uWfcsV/unW2dYzKjaEgZlVH1a17XogPCMfA+xoHK1+QBr5yXv5PNJNWkkAbXzNeIgpqj4ZL8xd5luld0qdV707HPNzo1cAWr6G8dt85QznB3DYi0No5ev11xyf3XAqLeMhNxvlMHJliuTiH41TE3YuboOUSYOj1J89ae8f4NJo=


In [37]:
// Now, similarly a verifier can use the same mechanism to verify that a given data is accurate

let verifier = crypto.createVerify('SHA256');
verifier.update(message)
console.log(verifier.verify(publicKey, signature, 'base64'))

true


In [35]:
// But the content of this message is not useful by itself, so the holder needs to attach the unhashed data

// For example, the holder can share the value for the name and and profession
const HOLDER_SHARED_DATA = {
    name: "Jack Sparrow",
    profession: "IT Manager"
}
// along with which hashing algorithm was used
const HOLDER_HASH_ALGORITHM = "SHA256"

In [56]:
// Now, a verifier can use this information to ensure that the unhashed data is correct.
// By rebuilding the hashed data object

// Step 0. verifier's own hash function
function hashValue(value: string, algorithm: string): string {
  const hash = crypto.createHash(algorithm);
  hash.update(value);
  return hash.digest('hex');
}

// Step 1. Retrieve data object from JSON data
const RETRIEVED_DATA = JSON.parse(message);

// step 2. Replace the respective values with the revealed data.
for (const key in HOLDER_SHARED_DATA) {
    RETRIEVED_DATA[key] = hashValue(HOLDER_SHARED_DATA[key], HOLDER_HASH_ALGORITHM);
}
RETRIEVED_DATA

// step 3. Make sure the reconstructed object is the same as the original hashed data
console.log(`Reconstructed data is the same as hashed data?: ${JSON.stringify(RETRIEVED_DATA) === message}`)

// Since we've already verified that the hashed message is valid in the previous code cell
// and now we verified that the hashed values are equal to the revealed values, then
// we conclude that we trust these revealed data.

Reconstructed data is the same as hashed data?: true


---

## Unlinkability

Unlinkability means that a verifier who interacts with a credential holder in an attempt to get access to some verifiable credentials, that verifiable should be unable to link different transactions made by the same holder using the same credential or different credentials belonging to the same holder.

In other words, the different verifications are **unlinkable** with respect to the verifier.

It's also important that the issuer of the credential doesn't have access to any information about when or how the issued credential is used.

### Is this cryptographic scheme unlinkable?

No! because verifiers can store the signature which will always stay the same. This way, verifiers can compare signatures, and deduce that it's the same credential by the same user. 

Another reason is that verifiers always have access to all the data in the hashed form, so even though they don't know the hashed value, they can still compare hashed values together, and deduce that result. For fields like age, it's also easy to guess the hidden value. 

In [59]:
// data in hashed form. hashing an age of 61 will always be the same.
RETRIEVED_DATA

{
  name: 'a5d0dd92108bc55cadfaf56e36a60a7121b059d9e0d09c2d24d853e9d1d14ea7',
  age: 'd029fa3a95e174a19934857f535eb9427d967218a36ea014b70ad704bc6c8d1c',
  profession: 'c249aac9d89f89626fbcaa519d53e386478a22b775414c324e61289e033f1652'
}
